In [13]:
import torch
import torch.nn.functional as F
import math

### Implementing Core attention mechanism

Attention(Q, K, V) = softmax(QKt/sqrt(d_k))V

In [7]:
batch_size = 32
context_len = 10
d_embed = 12288
d_k = 128

E = torch.randn(batch_size, context_len, d_embed) #tensor of embedded words
Wq = torch.randn(d_embed, d_k) #query matrix
Wk = torch.randn(d_embed, d_k) #key matrix
Wv = torch.randn(d_embed, d_k) #value matrix

E.shape, Wq.shape, Wk.shape

(torch.Size([32, 10, 12288]),
 torch.Size([12288, 128]),
 torch.Size([12288, 128]))

In [9]:
Q = E @ Wq
K = E @ Wk

Q.shape, K.shape

(torch.Size([32, 10, 128]), torch.Size([32, 10, 128]))

In [48]:
#creating attention pattern
AP = Q @ K.transpose(1, 2) #transpose along 1st and 2nd dim, leaving batch dim intact

K.transpose(1, 2).shape, AP.shape

(torch.Size([32, 128, 10]), torch.Size([32, 10, 10]))

In [56]:
#apply scale, mask, and softmax
AP /= math.sqrt(d_k) #scale

mask = torch.triu(torch.ones(AP.shape, dtype=torch.bool))
AP = AP.masked_fill(~mask, -float('inf')) #mask with -inf

AP = F.softmax(AP, dim=2)

In [ ]:
#matmul with value matrix
V = E @ Wv